In [1]:
import torch.nn as nn
import torch
import torch.nn.functional as F

In [3]:
k = torch.rand(100, 10)
M = torch.rand(100, 20, 10)
beta = torch.rand(100)
g = torch.rand(100)
w = torch.rand(100, 20)
gamma = torch.rand(100)
s = torch.rand(100, 3)
shifts = torch.tensor([-1, 0, 1])


def content_addressing(k, M, beta):
    sim = F.cosine_similarity(k.unsqueeze(1), M.unsqueeze(0), dim=-1)
    wc = F.softmax(beta.unsqueeze(1)*sim, dim=0)
    wc = wc.squeeze(0)
    return wc

def interpolation(g, wc, w):
    g = g.unsqueeze(1)
    wg = g*wc + (1 - g)*w
    return wg

def shift(wg, s, shifts):
    N = wg.shape[1]
    w_tilde = torch.zeros_like(wg)
    for i in range(N):
        for j in range(N):
            if (i - j) % N in shifts:
                w_tilde[:, i] = wg[:, j]*s[:, (shifts==((i - j) % N)).nonzero().squeeze()]
    return w_tilde


def sharpening(w_tilde, gamma):
    w_tilde = torch.pow(w_tilde, gamma.unsqueeze(1))
    wt = w_tilde/w_tilde.sum()   
    return wt 



In [5]:
wc = content_addressing(k, M, beta)
wg = interpolation(g, wc, w)
w_tilde = shift(wg, s, shifts)
wt = sharpening(w_tilde, gamma)
print(wt.shape)

torch.einsum('ij, ijk->ik', w, M).shape

torch.Size([100, 20])


torch.Size([100, 10])